In [20]:
import wrds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db = wrds.Connection(wrds_username='ericw553')
db

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [22]:
db.list_libraries()

['aha_sample',
 'ahasamp',
 'audit',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex_trial',
 'boardsmp',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compg',
 'compm',
 'compsamp',
 'compsamp_all',
 'compsamp_snapshot',
 'compseg',
 'contrib',
 'contrib_as_filed_financials',
 'contrib_ceo_turnover',
 'contrib_char_retur

In [23]:
db.create_pgpass_file()

In [24]:
db.describe_table(library='comp', table='fundq')

Approximately 2046844 rows in comp.fundq.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),Global Company Key
1,datadate,True,DATE,Data Date
2,fyearq,True,INTEGER,Fiscal Year
3,fqtr,True,SMALLINT,Fiscal Quarter
4,fyr,True,SMALLINT,Fiscal Year-end Month
...,...,...,...,...
642,mkvaltq,True,"NUMERIC(18, 4)",Market Value - Total
643,prccq,True,"NUMERIC(24, 12)",Price Close - Quarter
644,prchq,True,"NUMERIC(24, 12)",Price High - Quarter
645,prclq,True,"NUMERIC(24, 12)",Price Low - Quarter


In [25]:
db.describe_table(library='crsp',table='msf')

Approximately 5037353 rows in crsp.msf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),CUSIP Header
1,permno,True,INTEGER,PERMNO
2,permco,True,INTEGER,PERMCO
3,issuno,True,INTEGER,Nasdaq Issue Number
4,hexcd,True,SMALLINT,Exchange Code Header
5,hsiccd,True,INTEGER,Standard Industrial Classification Code
6,date,True,DATE,Date of Observation
7,bidlo,True,"NUMERIC(11, 5)",Bid or Low Price
8,askhi,True,"NUMERIC(11, 5)",Ask or High Price
9,prc,True,"NUMERIC(11, 5)",Price or Bid/Ask Average


In [48]:
start_date = '2016-01-01'
end_date = '2020-12-31'

query_crsp = """
    SELECT permno, date, prc, ret, shrout, vol
    FROM crsp.msf
    WHERE date BETWEEN '{}' AND '{}'
"""
msf_data = db.raw_sql(query_crsp.format(start_date, end_date))
msf_data['mcap'] = msf_data['prc'].abs() * msf_data['shrout']
msf_data['date'] = pd.to_datetime(msf_data['date']).dt.normalize().dt.strftime('%Y-%m')
msf_data

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wrds/sql.py:222: SyntaxWarning: invalid escape sequence '\p'
  """
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wrds/sql.py:248: SyntaxWarning: invalid escape sequence '\p'
  """
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wrds/sql.py:297: SyntaxWarning: invalid escape sequence '\:'
  passwd = passwd.replace(":", "\:")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wrds/sql.py:308: SyntaxWarning: invalid escape sequence '\:'
  oldline = line.replace("""\:""", "##COLON##")


OperationalError: (psycopg2.OperationalError) could not receive data from server: Operation timed out
SSL SYSCALL error: Operation timed out

[SQL: 
    SELECT permno, date, prc, ret, shrout, vol
    FROM crsp.msf
    WHERE date BETWEEN '2016-01-01' AND '2020-12-31'
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [27]:
query_fundq = """
    SELECT gvkey, datadate, rdq, fyearq, fqtr, atq, niq, saleq, epspxq, dlttq, cheq, ceqq
    FROM comp.fundq
    WHERE indfmt = 'INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C'
      AND datadate BETWEEN '{}' AND '{}'
"""

fundq_data = db.raw_sql(query_fundq.format(start_date, end_date))
fundq_data['datadate'] = pd.to_datetime(fundq_data['datadate']).dt.normalize().dt.strftime('%Y-%m')
fundq_data

,gvkey,datadate,rdq,fyearq,fqtr,atq,niq,saleq,epspxq,dlttq,cheq,ceqq
0,001410,2016-01,2016-03-08,2016,1.0,2255.200,14.000,1268.400,0.24,286.700,55.900,995.700
1,001562,2016-01,2016-02-25,2015,3.0,134.682,2.111,27.095,0.07,0.000,64.729,94.160
2,001618,2016-01,2016-03-15,2015,3.0,133.047,-0.569,12.198,-0.07,16.350,13.857,90.912
3,001632,2016-01,2016-02-17,2016,1.0,7598.081,164.504,769.429,0.53,1730.948,3789.468,4994.862
4,001633,2016-01,2016-03-02,2016,2.0,635.158,-2.985,127.869,-0.24,0.000,120.067,521.274
...,...,...,...,...,...,...,...,...,...,...,...,...
230518,348892,2020-12,2021-04-21,2020,4.0,17.474,-2.509,0.000,-0.22,0.000,13.705,15.631
230519,350681,2020-12,None,2020,4.0,8182.654,68.664,441.354,0.07,8.862,226.427,521.692
230520,351038,2020-12,None,2020,4.0,1.378,-2.095,0.000,-0.70,0.000,0.324,-6.607
230521,351491,2020-12,2021-10-29,2020,4.0,18841.000,28.000,4393.000,0.10,199.000,558.000,2734.000


In [28]:
db.describe_table(library='crsp', table='ccmxpf_linktable')

Approximately 87504 rows in crsp.ccmxpf_linktable.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),Standard and Poor\'\'s Identifier
1,linkprim,True,VARCHAR(1),Primary Link Marker
2,liid,True,VARCHAR(3),Security-level Identifer
3,linktype,True,VARCHAR(2),Link Type Code
4,lpermno,True,DOUBLE PRECISION,Historical CRSP PERMNO Link to COMPUSTAT Record
5,lpermco,True,DOUBLE PRECISION,Historical CRSP PERMCO Link to COMPUSTAT Record
6,usedflag,True,DOUBLE PRECISION,Flag marking whether link is used in building ...
7,linkdt,True,DATE,First Effective Date of Link
8,linkenddt,True,DATE,Last Effective Date of Link


In [29]:
query_link = """
    SELECT gvkey, lpermno AS permno, linkdt, linkenddt, linktype, linkprim
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LU', 'LC') AND linkprim IN ('C', 'P')
"""
link_table = db.raw_sql(query_link)
link_table['linkenddt'] = pd.to_datetime(link_table['linkenddt'].fillna(pd.Timestamp.today())).dt.normalize().dt.strftime('%Y-%m')
link_table['linkdt'] = pd.to_datetime(link_table['linkdt']).dt.normalize().dt.strftime('%Y-%m')
link_table

,gvkey,permno,linkdt,linkenddt,linktype,linkprim
0,001000,25881.0,1970-11,1978-06,LU,P
1,001001,10015.0,1983-09,1986-07,LU,P
2,001002,10023.0,1972-12,1973-06,LC,C
3,001003,10031.0,1983-12,1989-08,LU,C
4,001004,54594.0,1972-04,2025-02,LU,P
...,...,...,...,...,...,...
32490,349994,23514.0,2022-11,2025-02,LC,P
32491,350681,22205.0,2021-10,2023-03,LC,P
32492,351038,16161.0,2021-10,2025-02,LC,P
32493,352262,23773.0,2023-03,2025-02,LC,P


In [30]:
fundq_data = fundq_data.merge(link_table, on=['gvkey'] , how='inner')
fundq_data = fundq_data[(pd.to_datetime(fundq_data['datadate']) >= pd.to_datetime(fundq_data['linkdt'])) & (pd.to_datetime(fundq_data['datadate']) <= pd.to_datetime(fundq_data['linkenddt']))]
fundq_data

,gvkey,datadate,rdq,fyearq,fqtr,atq,niq,saleq,epspxq,dlttq,cheq,ceqq,permno,linkdt,linkenddt,linktype,linkprim
0,001410,2016-01,2016-03-08,2016,1.0,2255.200,14.000,1268.400,0.24,286.700,55.900,995.700,47730.0,1969-02,2025-02,LU,P
1,001562,2016-01,2016-02-25,2015,3.0,134.682,2.111,27.095,0.07,0.000,64.729,94.160,13777.0,1983-02,2025-02,LU,P
2,001618,2016-01,2016-03-15,2015,3.0,133.047,-0.569,12.198,-0.07,16.350,13.857,90.912,38746.0,1966-01,2025-02,LU,P
3,001632,2016-01,2016-02-17,2016,1.0,7598.081,164.504,769.429,0.53,1730.948,3789.468,4994.862,60871.0,1972-12,2025-02,LC,P
4,001633,2016-01,2016-03-02,2016,2.0,635.158,-2.985,127.869,-0.24,0.000,120.067,521.274,14198.0,1972-12,2018-06,LU,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153920,331856,2020-12,2021-02-26,2020,4.0,167.861,-11.159,0.000,-0.53,0.679,127.452,158.785,14615.0,2019-04,2025-02,LC,P
153921,332115,2020-12,2021-03-18,2020,4.0,39.516,-6.624,0.504,-0.36,10.877,9.649,18.857,80577.0,2019-05,2025-02,LC,P
153926,345556,2020-12,2021-03-29,2020,4.0,63.609,-8.547,0.163,-1.68,2.622,18.526,42.994,16069.0,2020-11,2023-03,LC,P
153929,345920,2020-12,2021-03-30,2020,4.0,275.795,-9.398,87.442,-0.30,15.610,76.955,210.918,20194.0,2020-12,2025-02,LC,P


In [31]:
msf_data['date'] = pd.to_datetime(msf_data['date'])
fundq_data['datadate'] = pd.to_datetime(fundq_data['datadate'])
fundq_data['linkdt'] = pd.to_datetime(fundq_data['linkdt'])
fundq_data['linkenddt'] = pd.to_datetime(fundq_data['linkenddt'])
msf_data['permno'] = pd.to_numeric(msf_data['permno'], errors='coerce').astype('Int64')
fundq_data['permno'] = pd.to_numeric(fundq_data['permno'], errors='coerce').astype('Int64')
fundq_data['rdq'] = pd.to_datetime(fundq_data['rdq'])
fundq_data['rdq_next'] = (fundq_data['rdq'] + pd.DateOffset(months=1)).replace(day=1)

combined_data = pd.merge_asof(
    msf_data,
    fundq_data,
    left_on='date', 
    right_on='rdq_next',
    by='permno',  # Ensures matching within each permno
    direction='backward'  # Matches the nearest previous quarterly report
)
combined_data

,permno,date,prc,ret,shrout,vol,mcap,gvkey,datadate,rdq,...,niq,saleq,epspxq,dlttq,cheq,ceqq,linkdt,linkenddt,linktype,linkprim
0,10001,2016-01-01,8.32000,0.116779,10505.0,4203.0,8.740160e+04,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
1,10025,2016-01-01,84.64000,0.100324,5107.0,8531.0,4.322565e+05,011903,2016-01-01,2016-03-10,...,7.895,253.553,1.55,210.335,11.678,96.101,1986-01-01,2017-01-01,LU,P
2,10026,2016-01-01,107.98000,-0.074484,18686.0,17153.0,2.017714e+06,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
3,10028,2016-01-01,0.40000,0.212121,12296.0,1630.0,4.918400e+03,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
4,10032,2016-01-01,34.95000,0.000859,33276.0,44347.0,1.162996e+06,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452748,93426,2020-12-01,31.48000,0.076239,12552.0,12527.0,3.951370e+05,185138,2020-12-01,2021-02-17,...,0.118,75.445,0.01,60.130,98.438,257.823,2010-07-01,2025-02-01,LC,P
452749,93427,2020-12-01,77.59000,0.135851,36938.0,40249.0,2.866019e+06,179583,2020-12-01,2021-02-01,...,35.384,453.827,0.96,38.173,488.440,1036.670,2010-06-01,2025-02-01,LC,P
452750,93429,2020-12-01,93.12000,0.019711,108042.0,167295.0,1.006087e+07,184500,2020-12-01,2021-02-05,...,87.300,844.200,0.81,1267.300,1149.900,3348.900,2010-06-01,2025-02-01,LC,P
452751,93434,2020-12-01,2.93000,0.122605,34165.0,21454.0,1.001035e+05,184259,2020-12-01,2021-02-11,...,-8.466,15.052,-0.25,16.699,2.926,68.571,2010-06-01,2025-02-01,LC,P


In [32]:
# Query CRSP stocknames to get tickers
query_ticker = """
    SELECT permno, namedt, nameenddt, ticker
    FROM crsp.stocknames
"""
stocknames = db.raw_sql(query_ticker)

# Map tickers to the combined data
combined_data = combined_data.merge(stocknames.drop_duplicates(subset=['permno']), on='permno', how='left')
combined_data['permno_copy'] = combined_data['permno']
combined_data = combined_data.sort_values(by=('date'), ascending=True)
combined_data = combined_data.groupby('permno_copy').ffill()


In [33]:
combined_data

,permno,date,prc,ret,shrout,vol,mcap,gvkey,datadate,rdq,...,dlttq,cheq,ceqq,linkdt,linkenddt,linktype,linkprim,namedt,nameenddt,ticker
0,10001,2016-01-01,8.32000,0.116779,10505.0,4203.0,8.740160e+04,NaN,NaT,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1986-01-09,1993-11-21,GFGC
4902,86763,2016-01-01,18.25000,-0.191046,10627.0,26231.0,1.939428e+05,NaN,NaT,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1999-03-26,2001-08-20,ABTL
4901,86755,2016-01-01,104.13000,-0.069104,362800.0,10773657.0,3.777836e+07,NaN,NaT,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1999-03-10,2004-11-30,QQQ
4900,86754,2016-01-01,13.75000,0.015283,22515.0,13674.0,3.095812e+05,NaN,NaT,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1999-03-17,2006-10-01,MUE
4899,86745,2016-01-01,7.12000,-0.232759,524211.0,1737869.0,3.732382e+06,NaN,NaT,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,1999-02-04,2008-04-03,RIMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447330,17701,2020-12-01,69.29000,0.194038,122248.0,331700.0,8.470564e+06,033275,2020-10-01,2020-12-07,...,52.727,420.443,528.582,2018-04-01,2025-02-01,LC,P,2018-04-27,2023-12-29,SMAR
447329,17700,2020-12-01,106.56000,0.105165,147803.0,220064.0,1.574989e+07,NaN,NaT,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,2018-04-26,2021-03-30,CDAY
447328,17699,2020-12-01,-33.87500,0.028918,50.0,14.0,1.693750e+03,NaN,NaT,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,2018-04-10,2021-07-27,VETS
447341,17722,2020-12-01,76.12000,0.034499,3000.0,1596.0,2.283600e+05,NaN,NaT,NaN,...,NaN,NaN,NaN,NaT,NaT,NaN,NaN,2018-04-09,2018-06-03,CZA


In [34]:
listed_req = combined_data[(pd.to_datetime(combined_data['linkenddt']) - pd.to_datetime(combined_data['linkdt'])).dt.days > 700]

listed_req = listed_req[listed_req['prc'] > 0]
listed_req = listed_req.sort_values(by='date')

listed_req = listed_req.dropna(subset=['ret', 'ticker'])

listed_req['date'] = pd.to_datetime(listed_req['date'])

# Sort data by date and market cap in descending order
listed_req_sorted = listed_req.sort_values(by=['date', 'mcap'], ascending=[True, False])

In [35]:
listed_req_sorted

,permno,date,prc,ret,shrout,vol,mcap,gvkey,datadate,rdq,...,dlttq,cheq,ceqq,linkdt,linkenddt,linktype,linkprim,namedt,nameenddt,ticker
3198,55976,2016-01-01,66.360,0.082545,3201893.0,2646657.0,2.124776e+08,011259,2016-01-01,2016-02-18,...,44030.000,8705.000,80546.000,1972-11-01,2025-02-01,LU,P,1972-11-20,2002-01-01,WMT
3369,66181,2016-01-01,125.760,-0.049074,1267881.0,1470501.0,1.594487e+08,005680,2016-01-01,2016-02-23,...,20888.000,2216.000,6316.000,1981-09-01,2025-02-01,LC,P,1981-09-22,1984-04-18,HOMD
3594,76076,2016-01-01,23.790,-0.116185,5052000.0,8289336.0,1.201871e+08,020779,2016-01-01,2016-02-10,...,21591.000,60375.000,60871.000,1990-02-01,2025-02-01,LC,P,1990-02-16,2000-07-31,CSCO
3262,60097,2016-01-01,75.920,-0.013001,1406156.0,1330889.0,1.067554e+08,007228,2016-01-01,2016-03-01,...,33681.000,17286.000,50816.000,1972-12-01,2025-02-01,LC,P,1972-12-14,1977-11-20,MDTR
4365,82654,2016-01-01,51.240,-0.033458,1486558.0,352072.0,7.617123e+07,015633,2016-01-01,2016-02-24,...,9854.000,238820.000,63111.000,1995-10-01,2025-02-01,LC,P,1995-10-16,2001-04-01,RY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447854,18468,2020-12-01,4.800,-0.128857,972.0,5324.0,4.665600e+03,284353,2020-12-01,2021-04-01,...,218.588,206.156,-19.481,2019-03-01,2025-02-01,LC,P,2019-03-27,2023-12-29,GNFT
448278,19129,2020-12-01,2.605,0.042000,1786.0,22412.0,4.652530e+03,035583,2020-12-01,2021-03-30,...,47.111,4.171,-11.008,2020-01-01,2025-02-01,LC,P,2020-01-17,2023-12-29,MDIA
451644,90432,2020-12-01,4.950,0.195652,793.0,5321.0,3.925350e+03,258664,2020-12-01,2021-04-23,...,11645.497,4429.422,3043.722,2004-11-01,2025-02-01,LC,P,2004-11-01,2011-06-09,BCA
447158,17380,2020-12-01,1.090,-0.148437,1818.0,8499.0,1.981620e+03,033161,2020-12-01,2021-03-18,...,105.476,194.546,-184.344,2018-03-01,2025-02-01,LC,P,2018-03-23,2019-04-03,STG


In [36]:


# Initialize an empty DataFrame to store the top 50 stocks across all dates
accumulated_top_50 = pd.DataFrame()

# Iterate over the unique dates and accumulate top 50 stocks for each date
for date in listed_req_sorted['date'].unique():
    # Get the stocks for the current date
    current_date_data = listed_req_sorted[listed_req_sorted['date'] == date]
    
    # Select top 50 stocks for the current date
    top_50_current_date = current_date_data.head(50)
    
    # Accumulate the top 50 stocks across dates, ensuring no duplicates
    accumulated_top_50 = pd.concat([accumulated_top_50, top_50_current_date]).drop_duplicates(subset=['permno'])

# Now, accumulated_top_50 contains the top 50 stocks across all dates, accumulated as the dates progress
top_tickers = accumulated_top_50['ticker'].unique()

listed_req['vol'] = listed_req.groupby('permno')['vol'].bfill()
listed_req = listed_req[listed_req['ticker'].isin(top_tickers)]
listed_req = listed_req.dropna()

In [39]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [47]:
listed_req[(listed_req['date']=='2016-01-01') | (listed_req['date']=='2016-02-01') | (listed_req['date']=='2016-03-01')].sort_values(by=['date', 'permno'], ascending=[True, True])

,permno,date,prc,ret,shrout,vol,mcap,gvkey,datadate,rdq,fyearq,fqtr,atq,niq,saleq,epspxq,dlttq,cheq,ceqq,linkdt,linkenddt,linktype,linkprim,namedt,nameenddt,ticker
196,12076,2016-01-01,116.00000,-0.060393,79532.0,318158.0,9.225712e+06,015520,2016-01-01,2016-03-24,2015.0,4.0,6474.400,271.900,2392.600,3.43,1328.700,137.700,3060.700,2008-09-01,2025-02-01,LC,P,2008-09-11,2023-12-29,SIG
803,13379,2016-01-01,46.29000,-0.212889,131543.0,401178.0,6.089125e+06,170616,2016-01-01,2016-02-25,2015.0,4.0,1536.839,-79.323,220.024,-0.61,0.000,1009.039,859.414,2012-04-01,2025-02-01,LC,P,2012-04-19,2023-12-29,SPLK
859,13511,2016-01-01,149.49001,-0.151300,85909.0,373378.0,1.284254e+07,171067,2016-01-01,2016-02-25,2016.0,2.0,2316.600,-57.300,334.700,-0.66,0.000,964.600,592.800,2012-07-01,2025-02-01,LC,P,2012-07-20,2016-08-21,PANW
936,13628,2016-01-01,63.01000,-0.209212,112000.0,442814.0,7.057120e+06,015044,2016-01-01,2016-02-29,2015.0,4.0,2730.094,-81.128,323.427,-0.42,507.476,1969.459,1136.157,2012-10-01,2025-02-01,LC,P,2012-10-12,2013-12-02,WDAY
1155,13957,2016-01-01,26.27000,-0.125208,200040.0,483719.0,5.255051e+06,017812,2016-01-01,2016-03-15,2015.0,4.0,6016.000,871.000,1584.000,4.91,4302.000,269.000,744.000,2013-06-01,2020-12-01,LC,P,2013-06-27,2020-12-23,HDS
1701,14702,2016-01-01,17.65000,-0.054633,1130725.0,3649661.0,1.995730e+07,001704,2016-01-01,2016-02-18,2016.0,1.0,13326.000,286.000,2257.000,0.25,3343.000,3116.000,7168.000,1972-12-01,2025-02-01,LU,P,1972-12-14,1986-03-31,AMAT
2503,15707,2016-01-01,13.76000,-0.094737,1742520.0,2745646.0,2.397708e+07,026156,2016-01-01,2016-03-03,2016.0,1.0,77352.000,267.000,8532.000,0.21,15229.000,8505.000,31249.000,2015-11-01,2025-02-01,LC,P,2015-11-02,2019-07-10,HPE
2788,21742,2016-01-01,120.74000,0.004234,39051.0,69341.0,4.715018e+06,002807,2016-01-01,2016-03-07,2015.0,3.0,2670.549,38.099,1565.940,0.98,830.466,53.563,1043.130,1983-10-01,2025-02-01,LU,P,1983-10-20,1986-03-31,CASY
2906,29946,2016-01-01,97.84000,-0.014505,116819.0,138284.0,1.142957e+07,002435,2016-01-01,2016-03-02,2015.0,3.0,4333.000,189.000,809.000,0.94,1229.000,317.000,1417.000,1962-07-01,2025-02-01,LU,P,1962-07-02,1968-01-01,BFD
2936,32870,2016-01-01,80.41000,0.020486,264229.0,350230.0,2.124665e+07,005709,2016-01-01,2016-02-16,2016.0,1.0,6040.889,235.061,2292.672,0.88,250.000,375.215,4160.288,1962-07-01,2025-02-01,LU,P,1962-07-02,1968-01-01,HRL
